In [19]:
from PIL import Image
import numpy as np

import torch
import torch.nn
import torch.optim as optim
from torchvision import transforms, models

import network.styler as StyleNet

import utils.func as func


import clip
import torch.nn.functional as F
from utils.text_templates import imagenet_templates

import random

from PIL import Image 
import PIL 
from torchvision import utils as vutils
import argparse
from torchvision.transforms.functional import adjust_contrast

from matplotlib import pyplot as plt

In [20]:
topil = transforms.ToPILImage()
topic = transforms.ToTensor()

In [21]:
model,pre = clip.load('ViT-B/32', device="cuda")

In [22]:
score_func = torch.nn.CosineSimilarity()

In [53]:
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm

def calculate_fid(act1, act2):
    print(act1.shape)
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis= 0), cov(act1, rowvar= False)
    mu2, sigma2 = act2.mean(axis= 0), cov(act2, rowvar= False)

    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)

    # calculate sqrt of product between cov
    covmean = sigma1.dot(sigma2)**0.5

    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
        # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0*covmean)
    return fid

In [55]:
def get_score(path1, path2):
    pil1 = Image.open(path1)
    p1 = pre(pil1).unsqueeze(0).to("cuda")
    v1 = model.encode_image(p1)

    pil2 = Image.open(path2)
    p2 = pre(pil2).unsqueeze(0).to("cuda")
    v2 = model.encode_image(p2)

    res = score_func(v1,v2)

    v1 = torch.cat([v1,v1],dim=0)
    v2 = torch.cat([v2,v2],dim=0)
    fid = calculate_fid(v1.cpu().detach().numpy(),v2.cpu().detach().numpy())

    return res.item(),fid

In [25]:
path1 = "../result/result01.jpg"
path2 = "../source_pic/emma.jpg"

In [56]:
res = get_score(path1, path2)
print(res)

(2, 512)
(0.89892578125, 20.734375)
